# GridLAB-D GeoData Subcommand

As of version 4.2.21, HiPAS GridLAB-D supports the handling of geographic data.  This command checks the version of GridLAB-D.

In [1]:
!gridlabd --version

HiPAS GridLAB-D 4.2.21-210511


The `geodata` subcommand uses the general syntax `gridlabd geodata OPTIONS DIRECTIVE [ARGUMENTS]`.  

## Creating location data

There are two basic types of geodata entities in GridLAB-D: 

1. an unordered collection of points each specified by a `latitude,longitude` tuple; and 

1. an ordered series of waypoints along a path specified by a sequence of `latitude,longitude` tuples.  

The interpretation of a the entity is left to the dataset processor, but it can often been specified using the `location` or `position` keys, respectively.  A geodata entity can be converted from one to another by simply changing the key (see Geodata Indexing below).

The `create` directive is used to create a new geodata entity.  The general syntax is `gridlabd geodata create LOCATIONS ...`.

There are two methods of introducing locations.  The first method introduces one or more `latitude,longitude` tuples directly in the command line.  For example, the following command creates a dataset with the approximate location of SLAC's main gate.

In [2]:
!gridlabd geodata create 37.415,-122.201

id,latitude,longitude
0,37.415,-122.201


Multiple locations can be introduced by adding them to the command, for example:

In [3]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


The second method uses an input file with locations and associated data, such as this example CSV file:

In [4]:
!head -n 4 path_example.csv

latitude,longitude,configuration,pole_height
37.415045141688054,-122.2056472090359,flat3,15.0
37.414698020593065,-122.20848749028133,sideT,15.0
37.414454093051745,-122.21044282065421,sideT,15.0


To use this file, the following command can be used:

In [5]:
!gridlabd geodata create path_example.csv | head -n 4

id,latitude,longitude,configuration,pole_height
0,37.41504514168805,-122.20564720903592,flat3,15.0
1,37.414698020593065,-122.20848749028131,sideT,15.0
2,37.414454093051745,-122.21044282065421,sideT,15.0


Note that if multiple locations are provided they are sequences in the order in which they are presented, including if locations are provided directly on the command line or from data files.

# Geodata formatting

The output format can be changed using the `-f` or `--format` option. The valid formats are `CSV`, `JSON`, `RAW`, and `TABLE`.  `CSV` format is the default, but can be specified as follows:

In [6]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f CSV

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


JSON output looks like this:

In [7]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f JSON

[{"latitude":37.41,"longitude":-122.201},{"latitude":37.42,"longitude":-122.201}]


RAW output is generated as follows

In [8]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f RAW

37.41 -122.201
37.42 -122.201


TABLE output is generated for easy reading:

In [9]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f TABLE

    latitude  longitude
id                     
0      37.41   -122.201
1      37.42   -122.201


Output formats may include an ordered field list, such as

In [10]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -f CSV:longitude,latitude

id,longitude,latitude
0,-122.201,37.41
1,-122.201,37.42


# Geodata indexing

The output can be indexed using one of several standards keys, or keys from the data. The default key is `id` which is the row number, as seen above. This key can be explicitly specified as follows:

In [11]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k id

id,latitude,longitude
0,37.41,-122.201
1,37.42,-122.201


The `location` key generates a geohash code:

In [12]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k location

location,latitude,longitude,id
9q9hg4rcn36n,37.41,-122.201,0
9q9hghpgq3d4,37.42,-122.201,1


The `position` key generates a distance index, treating the data rows as a series of waypoints along a path. When this key is used, the distance and heading also generated.

In [13]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k position

position,latitude,longitude,id,distance,heading
0,37.41,-122.201,0,0.0,
1111,37.42,-122.201,1,1111.9508023359672,0.0


Any field or set of fields may be used for indexing, e.g.,

In [14]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -k latitude,longitude

latitude,longitude,id
37.41,-122.201,0
37.42,-122.201,1


# Path waypoints

Waypoints can be generated along a path using the `-r` or `--resolution` option.  For example, 250-meter waypoints are generated using the following syntax:

In [26]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -r 250

position,latitude,longitude,id,distance,heading
0,37.41,-122.201,0.0,0.0,
250,37.412248300909305,-122.201,,250.0,0.0
500,37.41449660181861,-122.201,,500.0,0.0
750,37.41674490272792,-122.201,,750.0,0.0
1000,37.41899320363723,-122.201,,1000.0,0.0
1111,37.42,-122.201,1.0,1111.9508023359672,0.0


When using waypoints, the key is automatically set to `position`.  The use of other keys is not supported. In addition, any waypoint added in the process is not assigned a row `id` in order to protect the original row ids. 

The index can be changed with the `-k` or `--key` options using a pipe, e.g.,

In [32]:
!gridlabd geodata create 37.410,-122.201 37.420,-122.201 -r 250 | gridlabd geodata create -k location

location,position,latitude,longitude,id,distance,heading
9q9hg4rcn36n,0,37.41,-122.20100000000001,0.0,0.0,
9q9hg4xynkd4,250,37.412248300909305,-122.20100000000001,,250.0,0.0
9q9hg5pgnrdj,500,37.41449660181861,-122.20100000000001,,500.0,0.0
9q9hg5xbq6f1,750,37.41674490272792,-122.20100000000001,,750.0,0.0
9q9hg5zvqmfh,1000,37.41899320363723,-122.20100000000001,,1000.0,0.0
9q9hghpgq3d4,1111,37.42,-122.20100000000001,1.0,1111.9508023359672,0.0


# Datasets

The `merge` directive is used to merge a dataset with an existing geodata entity.  The general syntax is `gridlabd geodata merge -D DATASET OPTIONS

## Distance

The `distance` dataset provide great-circle distance calculations.  For example, the following calculated the distance from the first point in the series.

In [15]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,593.0


The `distance` dataset support different units, include `meters` or `m`, `kilometers` or `km`, `feet` or `ft`, `yards` or `yd`, `miles` or `mi`.  For example, the following calculates the distances in feet:

In [16]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201 --units=feet

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,1944.0


Note that the default precision with which distances are calculated in 0 decimals. Thus the distance in `km` is rounded up to `1.0`:

In [17]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201 --units=km

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,1.0


The precision can be changed thus: 

In [18]:
!gridlabd geodata merge -D distance 37.410,-122.201 37.420,-122.201 --units=km --precision=2

id,latitude,longitude,distance
0,37.41,-122.201,0.0
1,37.42,-122.201,0.59


## Address

You can use the `address` dataset to perform address resolution operations, both to and from latitude and longitude.

To obtain the address at a location, use the following command

In [19]:
!gridlabd geodata merge -D address 37.415,-122.201

id,latitude,longitude,address
0,37.415,-122.201,"Stanford Linear Accelerator Center National Accelerator Laboratory, Sand Hill Road, Menlo Park, San Mateo County, California, 94028, United States"


To find the latitude and longitude of an address, use the command:

In [20]:
!gridlabd geodata merge -D address --reverse "2575 Sand Hill Road, Menlo Park CA"

id,index,address,longitude,latitude
0,0,"2575 Sand Hill Road, Menlo Park CA",-122.20117634090091,37.4154632


You can perform both operations to resolve the "official" address from an incomplete address:

In [21]:
!gridlabd geodata merge -D address --reverse "2575 Sand Hill Road, Menlo Park CA" | gridlabd geodata merge -D address

id,index,address,longitude,latitude
0,0,"Stanford Linear Accelerator Center National Accelerator Laboratory, Sand Hill Road, Menlo Park, San Mateo County, California, 94028, United States",-122.20117634090091,37.415463200000005


## Elevation


In [22]:
!gridlabd geodata create 37.415,-122.201 | gridlabd geodata merge -D elevation

id,latitude,longitude,elevation
0,37.415,-122.20100000000001,83.0


# Debugging

The `-d` or `--debug` option can be used to obtain details about why an error occurred.  For example, the following command has an invalid `lat,lon` tuple and results in a simple error message:

In [23]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --units=furlongs

ERROR [geodata/distance]: unit 'furlongs' is not valid


Using the debug option provides a more detailed traceback of the error that can be helpful in diagnostic the problem, particularly when it originates in a dataset package.

In [24]:
!gridlabd geodata merge -D distance 37,-122 38,-122 --units=furlongs -d

Traceback (most recent call last):
  File "/usr/local/opt/gridlabd/4.2.21-210511-develop_geodata_subcommand/bin/gridlabd-geodata", line 794, in merge
    result = MODULE.apply(data,options,config,warning)
  File "/usr/local/opt/gridlabd/4.2.21-210511-develop_geodata_subcommand/share/gridlabd/geodata_distance.py", line 103, in apply
    raise Exception(f"unit '{options['units']}' is not valid")
Exception: unit 'furlongs' is not valid
DEBUG [geodata/distance]: merge(args='['37,-122', '38,-122', '--units=furlongs']') --> data = 
----
None
----
options = {
    "plot": {
        "figsize": [
            7.0,
            5.0
        ],
        "cmap": "",
        "categorical": false,
        "legend": false,
        "scheme": "",
        "k": 5,
        "vmin": NaN,
        "vmax": NaN,
        "aspect": "auto"
    },
    "show": {
        "block": true
    },
    "savefig": {
        "dpi": 300,
        "facecolor": "w",
        "edgecolor": "k",
        "orientation": "landscape",
       